In [1]:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python/caffe/proto/')
import caffe_pb2
from google.protobuf import text_format
model_dir='/home/old-ufo/caffe-train/vgg-cao-bn/'

original_proto_fname="vgg_cao_train_val.prototxt"
bnned_proto_fname="bn_after_"+original_proto_fname

original_proto = caffe_pb2.NetParameter()
bnned_proto = caffe_pb2.NetParameter()

f = open(model_dir+original_proto_fname, "rb")
text_format.Merge(f.read(),original_proto)
f.close()
bnned_layer_names=[];

for layer in original_proto.layer:
    new_layer = bnned_proto.layer.add();
    new_layer.CopyFrom(layer);
    i=0
    for b in new_layer.bottom:
        if b in bnned_layer_names:
            b=b+'_BN'
            new_layer.bottom[i]=b
        i+=1
    i=0
    for t in new_layer.top:
        if t in bnned_layer_names:
            t=t+'_BN'
            new_layer.top[i]=t
        i+=1
    if (new_layer.type == 'Convolution') or (new_layer.type=='InnerProduct') and 'clf' not in new_layer.name:
        bn_layer = bnned_proto.layer.add();
        bn_layer.name=new_layer.name+'_BN';
        bn_layer.type='BN'
        bn_layer.bottom.append(new_layer.name)
        bn_layer.top.append(bn_layer.name)
        bn_layer.bn_param.scale_filler.type='constant'
        bn_layer.bn_param.scale_filler.value=1.0000001
        bn_layer.bn_param.shift_filler.type='constant'
        bn_layer.bn_param.shift_filler.value=0.0000001
        param_w = bn_layer.param.add();
        param_w.lr_mult=1;
        param_w.decay_mult=1;
        param_b = bn_layer.param.add();
        param_b.lr_mult=2;
        param_b.decay_mult=0;
        bnned_layer_names.append(new_layer.name)
    
final_str=text_format.MessageToString(bnned_proto);

with open(model_dir+bnned_proto_fname, "w") as text_file:
    text_file.write(final_str)

In [3]:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python/caffe/proto/')
import caffe_pb2
from google.protobuf import text_format
model_dir='/home/old-ufo/caffe-train/vgg-cao-bn/'

original_proto_fname="vgg_cao_train_val.prototxt"
bnned_proto_fname="bn_after_"+original_proto_fname

original_proto = caffe_pb2.NetParameter()
bnned_proto = caffe_pb2.NetParameter()

f = open(model_dir+original_proto_fname, "rb")
text_format.Merge(f.read(),original_proto)
f.close()
bnned_layer_names=[];

for layer in original_proto.layer:
    new_layer = bnned_proto.layer.add();
    new_layer.CopyFrom(layer);
    i=0
    for b in new_layer.bottom:
        if b in bnned_layer_names:
            b=b+'_BN'
            new_layer.bottom[i]=b
        i+=1
    j=0
#    for t in new_layer.top:
#        if t in bnned_layer_names:
#            t=t+'_BN'
#            new_layer.top[i]=t
#        i+=1
    if (new_layer.type == 'PReLU'):
        bn_layer = bnned_proto.layer.add();
        bn_layer.name=new_layer.top[0]+'_BN';
        bn_layer.type='BN'
        bn_layer.bottom.append(new_layer.top[0])
        bn_layer.top.append(bn_layer.name)
        bn_layer.bn_param.scale_filler.type='constant'
        bn_layer.bn_param.scale_filler.value=1.0000001
        bn_layer.bn_param.shift_filler.type='constant'
        bn_layer.bn_param.shift_filler.value=0.0000001
        param_w = bn_layer.param.add();
        param_w.lr_mult=1;
        param_w.decay_mult=1;
        param_b = bn_layer.param.add();
        param_b.lr_mult=2;
        param_b.decay_mult=0;
        bnned_layer_names.append(new_layer.top[0])

    
final_str=text_format.MessageToString(bnned_proto);

with open(model_dir+bnned_proto_fname, "w") as text_file:
    text_file.write(final_str)